# SQL window functions

In this exercise, we will test our understanding and application of SQL window functions on a sample SQLite database file for a retail company called Northwind by performing complex calculations and analyses like ranking, running totals, and date differences. Ensure that you have downloaded the database file, Northwind.db.

## Learning objectives

By the end of this train, you should:
- Use the RANK() function to assign a ranking number to each row based on the order specified within the window.
- Use aggregate window functions to calculate running totals. 
- Use the LAG() function to help calculate the difference, in days, between consecutive date readings in our dataset.
- Use aggregate window functions to calculate the moving average. 

First, let's load our sample database:

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql


In [2]:
# Load the Northwind database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///Northwind.db
    

Here is a view of all of our tables in the database:

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Northwind_ERD.png"  style="width:500px";/>
<br>
<br>
    <em>Figure 1: Northwind ERD</em>
</div>

## Exercise

Run the necessary queries that will provide us with the following information. Compare your queries with the solutions at the end of this notebook.

### Exercise 1

Rank all the orders of a specific customer from the most recent to the least recent using window functions. Assume that the customer ID is `'ALFKI'`.

In [3]:
%%sql

SELECT name FROM sqlite_master WHERE type = 'table';

 * sqlite:///Northwind.db
Done.


[('Categories',),
 ('Customers',),
 ('Region',),
 ('Shippers',),
 ('Suppliers',),
 ('Employees',),
 ('EmployeeTerritories',),
 ('OrderDetails',),
 ('Orders',),
 ('Territories',),
 ('Products',)]

In [20]:
%%sql

SELECT 
    OrderID, 
    OrderDate,
    RANK() OVER(
        ORDER BY OrderDate) AS RankOrder
FROM Orders
WHERE 
    CustomerID = 'ALFKI'
LIMIT 10;

 * sqlite:///Northwind.db
Done.


[(10643, '1997-08-25 00:00:00', 1),
 (10692, '1997-10-03 00:00:00', 2),
 (10702, '1997-10-13 00:00:00', 3),
 (10835, '1998-01-15 00:00:00', 4),
 (10952, '1998-03-16 00:00:00', 5),
 (11011, '1998-04-09 00:00:00', 6)]

The `RANK()` window function is used here to rank each order of the customer with the ID `'ALFKI'` based on the `OrderDate`. The `DESC` keyword is used so that the most recent order gets the highest rank (i.e. 1).

### Exercise 2

Calculate a running total of the quantity of orders using window functions.

In [21]:
%%sql

SELECT 
    OrderID,
    Quantity,
    SUM(Quantity) OVER(
        ORDER BY OrderId) AS RunningTotal
FROM OrderDetails
GROUP BY OrderId
LIMIT 5;

 * sqlite:///Northwind.db
Done.


[(10248, 12, 12),
 (10249, 9, 21),
 (10250, 10, 31),
 (10251, 6, 37),
 (10252, 40, 77)]

The `SUM()` window function is used here to calculate a running total of the quantity of orders. The `ORDER BY` clause inside the `OVER()` clause ensures that the running total is calculated in the order of the `OrderID`. We then group our data by `OrderID`.

### Exercise 3


Use window functions to find the difference in successive order dates for each customer. **HINT:** The `TIMESTAMPDIFF()` function in MySQL is not available in SQLite. We can use the `julianday()` function to convert the dates to a floating point number and then calculate the difference.

In [36]:
%%sql

SELECT 
    OrderId,
    CustomerID,
    OrderDate,
    LAG(OrderDate, 1) OVER(
        PARTITION BY CustomerID
        ORDER BY OrderDate) AS PrevOrderDate,
    julianday(OrderDate) - 
    julianday(LAG(OrderDate) OVER(
        PARTITION BY CustomerId
        ORDER BY OrderDate)) AS DateDiff
FROM 
    Orders
LIMIT 10;

 * sqlite:///Northwind.db
Done.


[(10643, 'ALFKI', '1997-08-25 00:00:00', None, None),
 (10692, 'ALFKI', '1997-10-03 00:00:00', '1997-08-25 00:00:00', 39.0),
 (10702, 'ALFKI', '1997-10-13 00:00:00', '1997-10-03 00:00:00', 10.0),
 (10835, 'ALFKI', '1998-01-15 00:00:00', '1997-10-13 00:00:00', 94.0),
 (10952, 'ALFKI', '1998-03-16 00:00:00', '1998-01-15 00:00:00', 60.0),
 (11011, 'ALFKI', '1998-04-09 00:00:00', '1998-03-16 00:00:00', 24.0),
 (10308, 'ANATR', '1996-09-18 00:00:00', None, None),
 (10625, 'ANATR', '1997-08-08 00:00:00', '1996-09-18 00:00:00', 324.0),
 (10759, 'ANATR', '1997-11-28 00:00:00', '1997-08-08 00:00:00', 112.0),
 (10926, 'ANATR', '1998-03-04 00:00:00', '1997-11-28 00:00:00', 96.0)]

The `LAG()` window function is used twice here, once to get the previous order date for each customer, and again to calculate the difference between the current order date and the previous order date. The `PARTITION BY` clause is used to separate the data into partitions based on the `CustomerID`. 

Since the `TIMESTAMPDIFF()` function in MySQL is not available in SQLite, we use the `julianday()` function to convert the dates to a floating point number, which we can then use for subtraction to find the difference between two dates.

### Exercise 4

Calculate the moving average of the quantity of the last 3 orders for each product using window functions.

In [35]:
%%sql 

SELECT 
    OrderID,
    ProductID,
    Quantity,
    ROUND(AVG(Quantity) OVER (PARTITION BY ProductID ORDER BY OrderID ROWS BETWEEN 2 PRECEDING AND 0 FOLLOWING), 4) AS MovingAvgQuantity
FROM
    OrderDetails
ORDER BY 
    ProductID,
    OrderID
LIMIT 10;

 * sqlite:///Northwind.db
Done.


[(10285, 1, 45, 45.0),
 (10294, 1, 18, 31.5),
 (10317, 1, 20, 27.6667),
 (10348, 1, 15, 17.6667),
 (10354, 1, 12, 15.6667),
 (10370, 1, 15, 14.0),
 (10406, 1, 10, 12.3333),
 (10413, 1, 24, 16.3333),
 (10477, 1, 15, 16.3333),
 (10522, 1, 40, 26.3333)]

The `AVG()` window function is used here to calculate the moving average of `Quantity` for the last 3 orders (the current order and the two preceding orders) for each product. The window is defined using the `PARTITION BY` clause (to segment the data by `ProductID`) and the `ORDER BY` clause (to arrange the data in order of `OrderID`). The `ROWS BETWEEN` clause specifies the size and location of the window – in this case, **the current row and the two rows preceding it.**